In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from scipy.integrate import quad
import matplotlib.pyplot as plt
import torch.nn.init as init

class analytic_solution:
    def __init__(self, ra, r, lambd, mu, sigma):
        self.a = (1-ra)/ra * sigma**2
        self.b = 2 / ra * (ra * r - r - lambd)
        self.c = (lambd + r) ** 2 / (ra * sigma**2)
        self.g = -lambd*mu*(lambd + r) / (ra * sigma**2)
        self.nu = (self.b ** 2 - 4 * self.a * self.c) ** 0.5
        self.d = lambd*mu/ra
        self.r = r
        self.ra = ra
        self.lambd = lambd
        self.sigma = sigma
        self.mu = mu

    def integrand(self, t):
        a,d,r,ra,sigma,mu,lambd = self.a, self.d, self.r, self.ra, self.sigma, self.mu, self.lambd
        term1 = (a / 2) * (self.B(t) ** 2)
        term2 = d * self.B(t)
        term3 = (sigma ** 2 / 2) * self.C(t)
        term4 = ((lambd*mu) ** 2) / (2 * ra * sigma ** 2)
        term5 = r
        return term1 + term2 + term3 + term4 + term5


    def A(self, tau):
        A, _ = quad(self.integrand, 0, tau)
        return A

    def B(self, tau):
        b,nu,g,r = self.b, self.nu, self.g, self.r
        return (-4*g*r * (1-np.exp(-nu*tau/2))**2 +2*g*nu*(1-np.exp(-nu*tau))) / (nu * (2*nu - (b+nu)*(1-np.exp(-nu*tau))))


    def C(self, tau):
        b,c,nu = self.b, self.c, self.nu
        return (2*c*(1-np.exp(-nu*tau)))/(2*nu-(b+nu)*(1-np.exp(-nu*tau)))

    def optimal_nocost(self, tau, X):
        ra,lambd,mu,r,sigma = self.ra, self.lambd, self.mu, self.r, self.sigma
        return (1/ra) * ((lambd*(mu-X)/X)-r)/((sigma/X)**2) + (1-ra)/ra * (self.C(tau)*X + self.B(tau))*X

    def J(self, W, X, tau):
        ra = self.ra
        return ((W * (np.exp(self.A(tau) + self.B(tau) * X + (self.C(tau) * (X**2)) / 2)))**(1 - ra) - 1)/(1 - ra)

class BoundaryNetwork(nn.Module):

    # Define the neural network architecture -- think is correct, may need to change neuron count
    def __init__(self, input_dim, hidden_dim, output_dim=1):
        super(BoundaryNetwork, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_dim, hidden_dim * 2),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_dim * 2, hidden_dim * 4),
            nn.LeakyReLU(0.05),
            # nn.Sigmoid(),
            nn.Linear(hidden_dim * 4, output_dim)
        )
        self._init_weights()

    def _init_weights(self):
        for layer in self.model:
            if isinstance(layer, nn.Linear):
                init.xavier_uniform_(layer.weight, gain=0.01)  # Small gain to reduce initial output magnitude
                init.constant_(layer.bias, 0.0)  # Set biases to zero

    def forward(self, x):
        return self.model(x)

# Simulate an Ornstein-Uhlenbeck process -- think is correct
def simulate_ou_process(x0, mu, sigma, lam, dt, num_steps, num_paths):
    x = np.zeros((num_steps + 1, num_paths))
    x[0] = x0
    for t in range(1, num_steps + 1):
        dx = lam * (mu - x[t - 1]) * dt + sigma * np.sqrt(dt) * np.random.randn(num_paths)
        x[t] = x[t - 1] + dx
    return x

# Define the CRRA utility function -- think is correct
def crra_utility(w, gamma=2):
    small_number = 1
    w[w == 0] = small_number  # set 0s to small numbers to avoid issue
    if gamma == 1:
        return torch.log(w)
    else:
        return (w ** (1 - gamma) - 1) / (1 - gamma)

# Define the loss function -- think is correct
def loss_fn(terminal_wealth, starting_wealth, start_price, T, model):
    # print("Terminal wealth:", terminal_wealth, 'dimensions:', terminal_wealth.shape)
    # print("J(w,x,t):", model.J(starting_wealth, start_price, T), "U(w):", crra_utility(terminal_wealth))
    return torch.mean(model.J(starting_wealth, start_price, T) - crra_utility(terminal_wealth))

def update_wealth(W, pi_new, pi_old, price_change, beta):
    transaction_costs = beta * (pi_new - pi_old) ** (3/2)
    W_new = W + pi_new * price_change - transaction_costs
    return W_new
print('complete')

complete


In [ ]:
# Hyperparameters
num_epochs = 3000
learning_rate = 0.001
hidden_dim = 196
input_dim = 1  # Asset price, time
num_steps = 50  # Simulation steps
num_paths = 20000  # Simulation paths
x0, mu, sigma, lam = 13.95, 15.446, 0.606 * np.sqrt(252), 0.113 * 252
ra = 2  # Risk aversion
T = 1.0  # Time to maturity
dt = T / num_steps
# beta = 0.0001  # Transaction cost coefficient
beta = 0.01
r = 0.05  # Risk-free rate
w0 = 10000  # Initial wealth
h = dt
var = sigma**2 * (1 - np.exp(-2 * lam * h)) / (2 * lam)

# Ensure torch uses GPU
torch.set_default_tensor_type(torch.cuda.FloatTensor)
torch.set_printoptions(sci_mode=False)

# Initialize the analytic solution
analytic_model = analytic_solution(ra, r, lam, mu, sigma)

# Initialize models, optimizers
upper_net = BoundaryNetwork(input_dim, hidden_dim).to('cuda')
lower_net = BoundaryNetwork(input_dim, hidden_dim).to('cuda')
optimizer = optim.Adam(list(upper_net.parameters()) + list(lower_net.parameters()), lr=learning_rate)
# Define a learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=500, gamma=0.5)

# plotting data
average_wealths = []
losses = []
fig,ax = plt.subplots()
ax2 = ax.twinx()
ax.set_xlabel('Epoch')
ax.set_ylabel('Average Wealth', color='b')
ax2.set_ylabel('Loss', color='r')
final_wealths = []

for epoch in range(num_epochs):
    # Simulate price paths
    wealth = torch.ones((num_paths, 1), device='cuda') * w0
    x = torch.ones((num_paths, 1), device='cuda') * x0
    pi_minus = torch.zeros_like(wealth, device='cuda')  # Starting with no position
    y = torch.ones((num_paths, 1), device='cuda') * w0  # All wealth in riskless
    total_costs = torch.zeros_like(wealth, device='cuda')  # Initialize total costs
    t = 0
    for _ in range(num_steps):
        x = x.to(torch.float32)
        wealth = wealth.to(torch.float32)
        pi_minus = pi_minus.to(torch.float32)
        # inputs = torch.cat([x, wealth, pi_minus], dim=1)
        inputs = torch.cat([x], dim=1)

        no_cost_position = torch.minimum(torch.maximum(analytic_model.optimal_nocost(t * dt, x), torch.tensor(-1.0, device='cuda')), torch.tensor(1.0, device='cuda'))

        # Predict boundaries
        upper_boundary = no_cost_position + upper_net(inputs)
        upper_boundary = torch.maximum(torch.tensor(-1.0, device='cuda'), torch.minimum(upper_boundary, torch.tensor(1.0, device='cuda')))
        lower_boundary = no_cost_position - lower_net(inputs)
        lower_boundary = torch.minimum(torch.maximum(lower_boundary, torch.tensor(-1.0, device='cuda')), torch.tensor(1.0, device='cuda'))
        pi_plus = torch.min(upper_boundary, torch.max(lower_boundary, pi_minus))
        # pi_plus = torch.zeros_like(wealth, device='cuda')
        t = t + h
        # if (_ == num_steps/2) and epoch %100==0:
        #   print(f'time: {_}')
        #   print('pi_start', no_cost_position)
        #   print('pi_plus:', pi_plus)
        #   print('upper net', upper_net(inputs))
        #   print('lower net', lower_net(inputs))

        # Simulate OU process
        dx = mu - mu * np.exp(-lam * h) + np.exp(-lam * h) * x - x + torch.normal(0, torch.sqrt(torch.tensor(var, device='cuda')), (num_paths, 1))
        dy = (np.exp(r * h) - 1)
        # c = beta * (wealth * (torch.abs(pi_plus - pi_minus)))**(3 / 2)
        c = beta * (wealth * (torch.abs(pi_plus - pi_minus)))
        total_costs = total_costs + c
        dw = (pi_plus * wealth / x) * dx + (1 - pi_plus) * wealth * dy - c

        wealth = wealth + dw
        x = x + dx
        y = y * (1 + dy)
        pi_minus = (wealth - y) / wealth

    # Compute loss
    loss = loss_fn(wealth, w0, x0, T, analytic_model)

    losses.append(loss.item())
    average_wealths.append(torch.mean(wealth).item())
    final_wealths.extend(wealth.tolist())

    # Backpropagation and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Step the scheduler for learning rate decay
    scheduler.step()
    # Print progress
    if epoch % 100 == 0:
        current_lr = optimizer.param_groups[0]['lr']
        # print(pi_plus)
        print((f'Epoch [{epoch}/{num_epochs}], Loss: {loss.item():.10f}, LR: {current_lr:.6f}, Average Wealth: {torch.mean(wealth).item()}, Average total costs paid: {torch.mean(total_costs).item()}'))

print('Training finished.')

ax.plot(average_wealths, color='b')
ax2.plot(losses, color='r')
plt.show()


/home/br4250/.conda/envs/torch-env/lib/python3.12/site-packages/torch/__init__.py:1144: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at /opt/conda/conda-bld/pytorch_1729647378361/work/torch/csrc/tensor/python_tensor.cpp:432.)
  _C._set_default_tensor_type(t)


Epoch [0/3000], Loss: 0.0000415870, LR: 0.001000, Average Wealth: 27032.31640625, Average total costs paid: 7860.3037109375
Epoch [100/3000], Loss: 0.0000416864, LR: 0.001000, Average Wealth: 27017.95703125, Average total costs paid: 7832.96044921875
Epoch [200/3000], Loss: 0.0000416854, LR: 0.001000, Average Wealth: 26999.25, Average total costs paid: 7834.14208984375
Epoch [300/3000], Loss: 0.0000416788, LR: 0.001000, Average Wealth: 27026.24609375, Average total costs paid: 7831.857421875
Epoch [400/3000], Loss: 0.0000415371, LR: 0.001000, Average Wealth: 27021.94140625, Average total costs paid: 7844.33740234375
Epoch [500/3000], Loss: 0.0000417520, LR: 0.000500, Average Wealth: 26933.458984375, Average total costs paid: 7822.26611328125
Epoch [600/3000], Loss: 0.0000416579, LR: 0.000500, Average Wealth: 27007.654296875, Average total costs paid: 7831.490234375
Epoch [700/3000], Loss: 0.0000417300, LR: 0.000500, Average Wealth: 26962.181640625, Average total costs paid: 7820.496582